In [ ]:
# These will be in the requirements.txt
# !pip install scikit-learn
# !pip install statsmodels

In [ ]:
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.lines import Line2D
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
import statistics as stats

This Notebook takes the previously cleaned footfall dataset and uses Machine Learning to predict footfall.
The first step is to reshape the data so it can be used as a time series.

In [ ]:
df = pd.read_csv('../data/footfall_df_2.csv')

To deal with NULLS the two incomplete years are dropped

In [ ]:
df.drop('2004', axis=1, inplace=True)
df.drop('2025', axis=1, inplace=True)

In [ ]:
years_list = ['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018', '2019', '2020','2021','2022','2023','2024']

The aim of the next few cells is to rearrange the months vs years rows/columns to one single date column with the datatype being a date object to allow use of the datetime library.

In [ ]:
df= df.melt(id_vars=['Museum Name', 'Month'], value_vars=years_list, var_name='Year', value_name='Footfall')

In [ ]:
#inspect the melted rows
df.head(5)

In [ ]:
#concatenates the values of the two columns and format the resulting datetime object
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str), format='%Y-%m')

In [ ]:
df.head(5)

In [ ]:
#drops the columns no longer needed
df.drop('Year', axis=1, inplace=True)
df.drop('Month', axis=1, inplace=True)

In [ ]:
df.head(5)

In [ ]:
#remove the museum name column and group by date.
df = df.drop(columns=['Museum Name'], axis=1)
df = df.groupby('Date').sum()

In [ ]:
#resets the index for future use 
df = df.reset_index()

In [ ]:
#saves this timeseries ready to use for machine learning.
df.to_csv('../data/footfall_timeseries.csv', index=False)

In [ ]:
#check to see everything is as it should be
df.columns

## Machine Learning

This section of the notebook tests three different models.

### Classification of terms:
- A Decision Tree is a flowchart-like structure where each of the nodes represents a feature of the data, the branches are the rules and the final leaves are the outcomes of the algorithm.
- Ensemble Learning is where several models or methods are used in conjunction to facilitate better learning. Eg random forest.
- A random forest is simply several decision trees that are run on the same dataset but using different samples.  The outcome of each of these are averaged to achieve the final result.
- Boosting: Instead of running models in parallel, they are run sequentially so each iteratively improves on the last.
- Non-stationary time series: This is a dataset that contains trends.  Due to the increasing or decreasing mean over a time series, poorly chosen models can incorrectly conclude that time plays a larger part in the change.  A model not appropriate to non-stationary data will overfocus on the total for the time point and not the incremental change.

Test Data was prepared using a 80/20 split

In [ ]:
X = df[['Date']].astype(int)
y = df['Footfall']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

Linear Regression is a model in which a straight line is plotted to determine the predictive relationship between a datapoint and a target.     The limitations of this model versus the dataset were known from the outset – linear regression handles trends well but does not predict well based on datasets with upwards and downwards movement.  The reason this model was chosen was to simply provide a benchmark against which other models could be compared. 

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
prediction = reg.predict(X_test)

In [ ]:
df['Date']

In [ ]:
plt.plot(df['Date'], df['Footfall'])
date_snippet = df['Date'].iloc[-48:]
plt.plot(date_snippet, prediction)
plt.title('Footfall data vs Prediction')
plt.xlabel('Date')
plt.ylabel('Visitors (millions)')
plt.legend

In [ ]:
reg_2 = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

reg_2.fit(X_train, y_train)

reg_2_pred = reg_2.predict(X_test)


In [ ]:
plt.plot(df['Date'], df['Footfall'])
date_snippet = df['Date'].iloc[-48:]
plt.plot(date_snippet, prediction) # this is the orange line, linear regression
plt.plot(date_snippet, reg_2_pred) # this is the green line xgboost
plt.title('Footfall data vs Prediction')
plt.xlabel('Date')
plt.ylabel('Visitors (millions)')
plt.legend

In [ ]:
arima_model = ARIMA(y_train, order=(5, 1, 0))

arima_model_fit = arima_model.fit()

arima_pred = arima_model_fit.get_forecast(steps=48)
arima_values = arima_pred.predicted_mean

## Combined graphs

For illustraion of various predition methonds against actual data
Jo Higgs

In [ ]:
date_list = df['Date']
years_list = pd.to_datetime(years_list, format='%Y')

# palette_colors = {'Title': '#003f5c', 'Actual Visitor_Figures': '#2f4b7c', 'X-Axis-Label': '#665191', 'Y-Axis_label': '#a05195',
#                  'Acutual Figures guide': '#d45087', 'Grid2': '#f95d6a', 'Prediction1': '#ff7c43', 'Prediction2': '#ffa600'}

#palette_colors = {'Actual Visitor Figures': '#2f4b7c', 'actual figures guide': 'lightgrey', 'Linear Regression': '#ff7c43', 'xgboost': '#ffa600'}

custom_lines = [Line2D([0], [0], color='#2f4b7c', lw=4, label='Actual Visitor Figures'),
                Line2D([0], [0], color='lightgrey', lw=4, label='Actual Figures guide'),
                Line2D([0], [0], color='#ff7c43', lw=4, label='Linear Regression'),
                Line2D([0], [0], color='#ffa600', lw=4, label='XGBoost'),
                Line2D([0], [0], color='#f95d6a', lw=4, label='ARIMA')]

date_snippet = df['Date'].iloc[-48:]

fig, axs = plt.subplots(2, 2, layout='constrained', figsize = (20, 10))

ax = axs[0][0]
ax.plot(date_list, df['Footfall'], label='Actual Visitors', color='#2f4b7c')
ax.set_title('Actual Visitors', fontsize=20, color='#003f5c')
ax.set_xlabel('Date', fontsize=18, color='#003f5c')
ax.set_ylabel('Visitors (millions)', fontsize=14, color='#003f5c')
ax.set_ylim(0,4000000)
ax.tick_params(axis='x', rotation = 90)
ax.legend(custom_lines, ['Actual Visitors', 'Actual Visitors (guide)', 'Linear Regression', 'XGBoost', 'ARIMA'], loc='upper left')

ax = axs[0][1]
ax.plot(date_list, df['Footfall'], label='Actual Visitors', color='lightgrey', linestyle='--')
ax.plot(date_snippet, prediction, label='Linear Regression', color='#ff7c43')
ax.set_title('Linear Regression', fontsize=20, color='#003f5c')
ax.set_xlabel('Date', fontsize=18, color='#003f5c')
ax.set_ylabel('Visitors (millions)', fontsize=14, color='#003f5c')
ax.set_ylim(0,4000000) 
ax.tick_params(axis='x', rotation = 90)
ax.legend(custom_lines, ['Actual Visitors', 'Actual Visitors (guide)', 'Linear Regression', 'XGBoost', 'ARIMA'], loc='upper left')

ax = axs[1][0]
ax.plot(date_list, df['Footfall'], label='Actual Visitors', color='lightgrey', linestyle = '--')
ax.plot(date_snippet, reg_2_pred, label='XGBoost', color='#ffa600')
ax.set_title('XGBoost', fontsize=20, color='#003f5c')
ax.set_xlabel('Date', fontsize=18, color='#003f5c')
ax.set_ylabel('Visitors (millions)', fontsize=14, color='#003f5c')
ax.set_ylim(0,4000000)
ax.tick_params(axis='x', rotation = 90)
ax.legend(custom_lines, ['Actual Visitors', 'Actual Visitors (guide)', 'Linear Regression', 'XGBoost', 'ARIMA'], loc='upper left')

ax = axs[1][1]
ax.plot(date_list, df['Footfall'], label='Actual Visitors', color='lightgrey', linestyle = '--')
ax.plot(date_snippet, arima_values, label='ARIMA', color='#f95d6a')
ax.set_title('ARIMA', fontsize=20, color='#003f5c')
ax.set_xlabel('Date', fontsize=18, color='#003f5c')
ax.set_ylabel('Visitors (millions)', fontsize=14, color='#003f5c')
ax.set_ylim(0,4000000)
ax.tick_params(axis='x', rotation = 90)
ax.legend(custom_lines, ['Actual Visitors', 'Actual Visitors (guide)', 'Linear Regression', 'XGBoost', 'ARIMA'], loc='upper left')

plt.suptitle('Footfall data vs Prediction', color='#003f5c', fontsize='24')
fig.savefig('../visualisations/predictionSet1.png', orientation='landscape')

In [ ]:
ts = pd.read_csv('../data/footfall_timeseries.csv')

In [ ]:
ts.index

In [ ]:
ts_fold = TimeSeriesSplit(n_splits=5)

In [ ]:
ts.columns

In [ ]:
ts.index

In [ ]:
ts['Date'].dtypes

In [ ]:
ts['Date'] = pd.to_datetime(ts['Date'])

In [ ]:
ts['Quarter'] = ts['Date'].dt.quarter #keep as a datetime object don't convert to int
ts['Month'] = ts['Date'].dt.month
ts['Year'] = ts['Date'].dt.year

ts['lag_day'] = ts['Footfall'].shift(1)
ts['lag_week'] = ts['Footfall'].shift(7)
ts['lag_fortnight'] = ts['Footfall'].shift(14)
ts['lag_threeweeks'] = ts['Footfall'].shift(21)

ts['lag_fiveweeks'] = ts['Footfall'].shift(35)


In [ ]:
ts.head(5)

In [ ]:
ts = ts.dropna()

In [ ]:
#ts_X = ts[['Date']].astype(int) # needs to be a matrix of features, the 80/20 split 'faked' a feature due to future dates
ts_X = ts[['Quarter', 'Month', 'Year', 'lag_day', 'lag_week', 'lag_fortnight', 'lag_threeweeks', 'lag_fiveweeks']]
ts_y = ts['Footfall']

In [ ]:
print(ts_X)

In [ ]:
mae_list = []
rmse_list = []

for train_index, test_index in ts_fold.split(ts):
    
    ts_X_train, ts_X_test = ts_X.iloc[train_index], ts_X.iloc[test_index]
    ts_y_train, ts_y_test = ts_y.iloc[train_index], ts_y.iloc[test_index]

    model = xgb.XGBRegressor()
    model.fit(ts_X_train, ts_y_train)

    ts_pred = model.predict(ts_X_test)

    mae = mean_absolute_error(ts_y_test, ts_pred)
    mae_list.append(mae)
    rmse = np.sqrt(mean_squared_error(ts_y_test, ts_pred))
    rmse_list.append(rmse)
    

In [ ]:
print(mae_list, rmse_list)

In [ ]:

ave_mae = stats.mean(mae_list)
ave_rmse = stats.mean(rmse_list)

ts_mean = ts['Footfall'].mean()

print(ave_mae, ave_rmse, ts_mean)

mae_percentage = ave_mae / ts_mean
rmse_percentage = ave_rmse / ts_mean

print(mae_percentage, rmse_percentage)


In [ ]:
plt.plot(ts['Date'], ts['Footfall'])
date_snippet = ts['Date'].iloc[-34:]
plt.plot(date_snippet, ts_pred) 
plt.title('Footfall data vs Cross-validated & Feature-Engineering Prediction')
plt.xlabel('Date')
plt.ylabel('Visitors (millions)')
plt.legend

In [ ]:
#checking the initial error for xgb 
xgb_mae = mean_absolute_error(y_test, reg_2_pred)   
xgb_rmse = np.sqrt(mean_squared_error(y_test, reg_2_pred))
df_mean = df['Footfall'].mean()

mae_error = xgb_mae / ts_mean
rmse_error = xgb_rmse / ts_mean

print(mae_error, rmse_error)


In [ ]:
#retraining the new xgb on 80% dataset to compare.

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(ts_X, ts_y, test_size = 0.2, random_state=5)

In [ ]:
reg_final = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

reg_final.fit(X_train_final, y_train_final)

reg_pred_final = reg_final.predict(X_test_final)


In [ ]:
xgb_mae_final = mean_absolute_error(y_test_final[:41], reg_pred_final)   
xgb_rmse_final = np.sqrt(mean_squared_error(y_test[:41], reg_pred_final))
df_mean = df['Footfall'].mean()

mae_error_final = xgb_mae_final / ts_mean
rmse_error_final = xgb_rmse_final / ts_mean

print(mae_error_final, rmse_error_final)

0.08300221193240777 0.4459227471392752

In [ ]:
plt.plot(ts['Date'], ts['Footfall'])
date_snippet = ts['Date'].iloc[-41:]
plt.plot(date_snippet, reg_pred_final) 
plt.title('Footfall data vs Cross-validated & Feature-Engineering Prediction 80% training dataset')
plt.xlabel('Date')
plt.ylabel('Visitors (millions)')
plt.legend

## Footfall data vs Cross-validated & Feature-Engineering Prediction
Graph in same format as PredictionSet1

Jo Higgs

In [ ]:
date_list = df['Date']
years_list = pd.to_datetime(years_list, format='%Y')


custom_lines = [Line2D([0], [0], color='#2f4b7c', lw=4, label='Actual Visitor Figures'),
                Line2D([0], [0], color='lightgrey', lw=4, label='Actual Figures guide'),
                Line2D([0], [0], color='#ff7c43', lw=4, label='XGBoost (Trained on 20% dataset)'),
                Line2D([0], [0], color='#ffa600', lw=4, label='XGBoost (Trained on 80% dataset)')]

date_snippet1 = ts['Date'].iloc[-34:]
date_snippet2 = ts['Date'].iloc[-41:]

fig, axs = plt.subplots(1, 2, layout='constrained', figsize = (20, 10))

ax = axs[0]
ax.plot(date_list, df['Footfall'], label='Actual Visitors', color='lightgrey', linestyle='--')
ax.plot(date_snippet1, ts_pred, label='ts_predict', color='#ff7c43')
ax.set_title('XGBoost (Trained on 20% dataset)', fontsize=20, color='#003f5c')
ax.set_xlabel('Date', fontsize=18, color='#003f5c')
ax.set_ylabel('Visitors (millions)', fontsize=14, color='#003f5c')
ax.set_ylim(0,4000000) 
ax.tick_params(axis='x', rotation = 90)
ax.legend(custom_lines, ['Actual Visitors', 'Actual Visitors (guide)', 'XGBoost (Trained on 20% dataset)', 'XGBoost (Trained on 80% dataset)'], loc='upper left')

ax = axs[1]
ax.plot(date_list, df['Footfall'], label='Actual Visitors', color='lightgrey', linestyle = '--')
ax.plot(date_snippet2, reg_pred_final, label='XGBoost (Trained on 80% dataset)', color='#ffa600')
ax.set_title('XGBoost (Trained on 80% dataset)', fontsize=20, color='#003f5c')
ax.set_xlabel('Date', fontsize=18, color='#003f5c')
ax.set_ylabel('Visitors (millions)', fontsize=14, color='#003f5c')
ax.set_ylim(0,4000000)
ax.tick_params(axis='x', rotation = 90)
ax.legend(custom_lines, ['Actual Visitors', 'Actual Visitors (guide)', 'XGBoost (Trained on 20% dataset)', 'XGBoost (Trained on 80% dataset)'], loc='upper left')

plt.suptitle('Footfall data vs Cross-validated & Feature-Engineering Predictions', color='#003f5c', fontsize='24')
fig.savefig('../visualisations/predictionSet2.png', orientation='landscape')